# Luminar

## Minimal Code

In [1]:
from typing import Iterable, NamedTuple

import torch
from datasets import Dataset
from torch import nn
from torch.utils.data import DataLoader, Subset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    PreTrainedModel,
    PreTrainedTokenizer,
)

### Luminar Encoder

1. Pre-Process Inputs: tokenize and pass through LLM, recording hidden states
2. Calculate _Intermediate Likelihoods_: pass each hidden state through the models LM head

In [2]:
class LuminarEncoder:
    def __init__(
        self,
        feature_dim: int = 256,
        model_name_or_path: str = "gpt2",
        device: str = ("cuda" if torch.cuda.is_available() else "cpu"),
    ):
        self.feature_dim = feature_dim
        self.device = torch.device(device)

        self.tokenizer: PreTrainedTokenizer = AutoTokenizer.from_pretrained(
            model_name_or_path
        )
        if not hasattr(self.tokenizer, "pad_token") or self.tokenizer.pad_token is None:
            self.tokenizer.pad_token = self.tokenizer.eos_token
        self.pad_token_id = self.tokenizer.pad_token_id

        self.model: PreTrainedModel = AutoModelForCausalLM.from_pretrained(
            model_name_or_path
        )
        self.model = self.model.to(self.device)

        if hasattr(self.model, "lm_head"):
            self.model_lm_head: nn.Linear = self.model.lm_head
        elif hasattr(self.model.model, "lm_head"):
            self.model_lm_head: nn.Linear = self.model.model.lm_head
        else:
            raise ValueError("Could not find lm_head in model")

    def __call__(self, batch: dict[str, list[str]]) -> dict[str, list[torch.Tensor]]:
        return {"features": self.process(batch["text"])}

    def process(self, batch: list[str]) -> list[torch.Tensor]:
        encoding = self.tokenizer(
            batch,
            padding=True,
            truncation=True,
            max_length=self.feature_dim,
            return_tensors="pt",
        )
        batch_hidden_states = self.forward(encoding.input_ids, encoding.attention_mask)

        intermediate_likelihoods = []
        for input_ids, hidden_states in zip(encoding.input_ids, batch_hidden_states):
            intermediate_likelihoods.append(
                self.compute_intermediate_likelihoods(input_ids, hidden_states)
            )

        return intermediate_likelihoods

    @torch.no_grad()
    def forward(
        self,
        input_ids: torch.Tensor,
        attention_mask: torch.Tensor,
    ) -> Iterable[tuple[torch.Tensor, ...]]:
        outputs = self.model(
            input_ids=input_ids.to(self.device),
            attention_mask=attention_mask.to(self.device),
            output_hidden_states=True,
        )

        # unpack hidden states to get one list of tensors per input sequence,
        # instead of one hidden state per layer in the model
        return zip(*outputs.hidden_states)  # type: ignore

    @torch.no_grad()
    def compute_intermediate_likelihoods(
        self,
        input_ids: torch.Tensor,
        hidden_states: tuple[torch.Tensor],
    ) -> torch.Tensor:
        labels = input_ids[1:].view(-1, 1)

        seq_length = min(labels.ne(self.pad_token_id).sum(), self.feature_dim)
        labels = labels[:seq_length].to(self.device)

        intermediate_likelihoods = []
        for hs in hidden_states:
            hs: torch.Tensor = hs[:seq_length].to(self.device)
            il = (
                # get layer logits
                self.model_lm_head(hs)
                # calculate likelihoods
                .softmax(-1)
                # get likelihoods of input tokens
                .gather(-1, labels)
                .squeeze(-1)
                .cpu()
            )
            del hs

            # pad with zeros if sequence is shorter than required feature_dim
            if seq_length < self.feature_dim:
                il = torch.cat([il, torch.zeros(self.feature_dim - seq_length)])

            intermediate_likelihoods.append(il)

        # stack intermediate likelihoods to get tensor of shape (feature_dim, num_layers)
        return torch.stack(intermediate_likelihoods, dim=1)

### Luminar Classifier

CNN-based classifier using _Intermediate Likelihoods_ as input features.
Here, we utilize these inherently 2D values (`seq_len * num_layers`) as 1D inputs where the second dimension is treated as input channels.

In [3]:
class ConvolutionalLayerSpec(NamedTuple):
    channels: int
    kernel_size: int | tuple[int, int]
    stride: int = 1

    @property
    def kernel_size_1d(self):
        if isinstance(self.kernel_size, int):
            return self.kernel_size
        return self.kernel_size[0]

    @property
    def kernel_size_2d(self):
        if isinstance(self.kernel_size, int):
            return (self.kernel_size, self.kernel_size)
        return self.kernel_size

    @property
    def padding(self) -> int:
        return (self.kernel_size_1d - 1) // 2

    def __repr__(self):
        return repr(tuple(self))


DEFAULT_CONV_LAYER_SHAPES = ((64, 5), (128, 3), (128, 3), (128, 3), (64, 3))


class LuminarClassifier(nn.Module):
    def __init__(
        self,
        conv_layer_shapes: Iterable[ConvolutionalLayerSpec] = DEFAULT_CONV_LAYER_SHAPES,
        projection_dim: int | None = None,
    ):
        super().__init__()
        self.conv_layers = nn.Sequential()
        for conv in conv_layer_shapes:
            conv = ConvolutionalLayerSpec(*conv)
            self.conv_layers.append(
                nn.LazyConv1d(
                    conv.channels,
                    conv.kernel_size,  # type: ignore
                    conv.stride,
                    conv.padding,
                ),
            )
            self.conv_layers.append(
                nn.LeakyReLU(),
            )
        self.conv_layers.append(nn.Flatten())

        if projection_dim:
            self.projection = nn.Sequential(
                nn.LazyLinear(projection_dim), nn.LeakyReLU()
            )
        else:
            self.projection = nn.Identity()

        self.classifier = nn.LazyLinear(1)

    def forward(self, features: torch.Tensor):
        # We are using 2D features (so `features` is a 3D tensor)
        # but we want to treat the second feature dimension as channels.
        # Thus, we need to transpose the tensor here
        features = features.transpose(1, 2)

        for layer in self.conv_layers:
            features = layer(features)

        return self.classifier(self.projection(features.flatten(1)))


## Example

### Prepare Data

In [4]:
import bz2
import json

import os
from datasets import concatenate_datasets

dataset_list = []
data_dir = "/resources/public/stoeckel/PrismAI/data"

for fname in os.listdir(data_dir):
    if fname.endswith(".jsonl.bz2"):
        path = os.path.join(data_dir, fname)
        dataset = Dataset.from_list(
            [json.loads(line) for line in bz2.open(path, "rt")]
        )
        dataset_list.append(dataset)

raw_dataset = concatenate_datasets(dataset_list)

In [5]:
from collections import defaultdict


def flatten_samples(batch: dict):
    result = defaultdict(list)
    for element in batch["samples"]:
        for sample in element:
            for key, value in sample.items():
                result[key].append(value)
    return result


dataset = raw_dataset.train_test_split(test_size=0.2).map(
    flatten_samples,
    batched=True,
    remove_columns=raw_dataset.column_names,
)
dataset

Map:   0%|          | 0/74735 [00:00<?, ? examples/s]

Map:   0%|          | 0/18684 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['agent', 'label', 'label_str', 'text'],
        num_rows: 149470
    })
    test: Dataset({
        features: ['agent', 'label', 'label_str', 'text'],
        num_rows: 37368
    })
})

In [6]:
dataset['train'][0]

{'agent': 'human',
 'label': 0,
 'label_str': 'human',
 'text': 'I disagree with this decision because not everyone does nor likes to do activitys. Some people like to do activitys like sports and yearbook but to force all students to do an extracurricular activity is unfair to those who dont want to. It should be a choice whether or not you want to have a extracurricular activity or not. I say it should be a choice because alot of people dont like to be active and dont try. If a principal trys to force extracurricular activities on his\\her students than most students wont like nor participate in most of the activitys the classes will be doing once they are in the class.\n\nIf i where to agree with this decision than i would say that it would be a great idea for the people who do like to play sports, help with yearbook, and be on the student council. I would also say it would be a great opportunity for the students who dont like these activities because it would give them a chance to 

### Encode Samples

In [7]:
encoder = LuminarEncoder(128, model_name_or_path="gpt2")

In [8]:
dataset = dataset.map(encoder, batched=True, batch_size=128)

Map:   0%|          | 0/149470 [00:00<?, ? examples/s]

Map:   0%|          | 0/37368 [00:00<?, ? examples/s]

### Run Training

In [9]:
model = LuminarClassifier()
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters())

In [10]:
from tqdm import tqdm


train_dataset = dataset["train"].with_format("torch", ["features", "label"])
for batch in tqdm(
    DataLoader(train_dataset, 32)
):
    optimizer.zero_grad()
    features = batch["features"]
    labels = batch["label"].float().unsqueeze(-1)

    preds = model(features)

    loss = criterion(preds, labels)

    loss.backward()
    optimizer.step()

100%|██████████████████████████████████████████████████████████████████████████████████████████| 4671/4671 [02:42<00:00, 28.74it/s]


In [11]:
save_path = "luminar_classifier.pt"
torch.save(model.state_dict(), save_path)

print(f"Model saved to {save_path}")


Model saved to luminar_classifier.pt


In [12]:
import numpy as np
from sklearn.metrics import f1_score


y_pred, y_truth, losses = [], [], []
test_dataset = dataset["test"].with_format("torch", ["features", "label"])
for batch in tqdm(DataLoader(test_dataset, 32)):
    with torch.no_grad():
        features = batch["features"]
        labels = batch["label"].float().unsqueeze(-1)
        preds = model(features)

        y_pred.extend(preds.sigmoid().round().squeeze().tolist())
        y_truth.extend(labels.squeeze().tolist())

        loss = criterion(preds, labels)
        losses.append(loss.item())

print(f"loss={np.mean(losses)}")
print(f"f1={f1_score(y_truth, y_pred)}")

100%|██████████████████████████████████████████████████████████████████████████████████████████| 1168/1168 [00:35<00:00, 33.35it/s]

loss=0.14466811366433482
f1=0.9396291504959035


In [ ]:
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score


# Source: Ghostbuster, Verma et al. (2024)
def get_scores(labels, probabilities, calibrated=False, precision=6):
    assert len(labels) == len(probabilities)

    if calibrated:
        threshold = sorted(probabilities)[len(labels) - sum(labels) - 1]
    else:
        threshold = 0.5

    acc = round(float(accuracy_score(labels, probabilities > threshold)), precision)
    f1 = round(float(f1_score(labels, probabilities > threshold)), precision)

    if sum(labels) == 0 or sum(labels) == len(labels):
        auroc = -1
    else:
        auroc = round(float(roc_auc_score(labels, probabilities)), precision)

    return acc, f1, auroc